In [ ]:
# Adapted from Robert Guthrie https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html
# And: https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/
import sklearn
from sklearn.linear_model import LinearRegression
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import json
import zoib

torch.manual_seed(1)

In [ ]:
def split_train_test_val(df):
    ind_year = np.where(np.array(df.index.names)=='year')[0][0]
    train_df = df.loc[df.index.get_level_values(ind_year)<=2010]
    val_df = df.loc[(df.index.get_level_values(ind_year)>2010) & (df.index.get_level_values(ind_year)<=2014)]
    test_df = df.loc[df.index.get_level_values(ind_year)>2014]
    return train_df, val_df, test_df

# Load data

In [ ]:
# Set the number of playas from 1 to 8
num_playas = 1

In [ ]:
# Read csv and set to appropriate number of playas
traj = pd.read_csv('./prepped_8.csv')
traj = traj.loc[traj['new_id'].isin(range(num_playas))]
# Set indices
traj.set_index(['year','month','id'], inplace=True)

# Drop embedding new_id
# traj.drop(columns=['new_id'], inplace=True)

# Here's a chance to drop everything else, if you'd like
traj = traj[['precip', 'new_id','inundation']]
traj['precip'] = 0

# Prep data

In [ ]:
# Params to set
hidden_dim = 64

num_layers=1
learning_rate = 0.005
num_epochs = 100
weight_decay = 0
lr_gamma = 1.0
lr_decay_step_size = 10 # Set this high if not needed
regularization_weight = 0
batch_size = 2

early_stopping=100000 # Set high if not needed

In [ ]:
def split_by_id(x, y):
    seq_length = np.sum((x[:,-1]==x[0,-1]))
    seq_starts = np.arange(0, x.shape[0], seq_length)
    x_arr = np.array([np.array(x[i:(i+seq_length)]) for i in seq_starts])
    y_arr = np.array([np.array(y[i:(i+seq_length)]) for i in seq_starts])
    return x_arr, y_arr

In [ ]:
scaler = StandardScaler()
train, val, test = split_train_test_val(traj)
train_X, train_y = train.values[:, :-1], train.values[:, -1]
val_X, val_y = val.values[:, :-1], val.values[:, -1]
test_X, test_y = test.values[:, :-1], test.values[:, -1]
# Run scaler, but not on ID
# train_X[:,:-1] = scaler.fit_transform(train_X[:,:-1])
# val_X[:,:-1] = scaler.transform(val_X[:,:-1])
# test_X[:,:-1] = scaler.transform(test_X[:,:-1])

num_playas = int(train_X[:, -1].max())+1
lstm_input_size = traj.shape[1]-1

In [ ]:
# Split into arrays by id
train_X_array, train_y = split_by_id(train_X, train_y)
val_X_array, val_y = split_by_id(val_X, val_y)
test_X_array, test_y = split_by_id(test_X, test_y)


In [ ]:
train_ds = torch.utils.data.TensorDataset(torch.Tensor(train_X_array), torch.Tensor(train_y))
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=False)

val_ds = torch.utils.data.TensorDataset(torch.Tensor(val_X_array), torch.Tensor(val_y))
val_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=False)

In [ ]:
# Here we define our model as a class
class LSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, output_dim=1,
                    num_layers=1):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.num_playas = num_playas
        
        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers, batch_first=True)

        # Define activations for output
        self.linear = nn.Linear(self.hidden_dim, output_dim)
    

    def init_hidden(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        # Forward pass through LSTM layer
        # shape of input: [batch_size, timesteps, input_dims]
        # shape of lstm_out: [batch_size, timesteps, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        # Shape of y_pred: [batch_size, timesteps, 1]
        # Run ids through embedding layer        
        # Concat and run through LSTM
        # Initialize hidden and cell state
        
        h0 = torch.zeros(self.num_layers, input.size(0), self.hidden_dim)
        # Initialize cell state
        c0 = torch.zeros(self.num_layers, input.size(0), self.hidden_dim)
        
        assert h0.shape == torch.Size([self.num_layers, input.size(0), self.hidden_dim])
        assert c0.shape == torch.Size([self.num_layers, input.size(0), self.hidden_dim])

        
        lstm_out, (hn, cn) = self.lstm(input, (h0, c0))
        
        # Assert that shapes are still as expected
        assert hn.shape == torch.Size([self.num_layers, input.size(0), self.hidden_dim])
        assert cn.shape == torch.Size([self.num_layers, input.size(0), self.hidden_dim])
        assert lstm_out.shape == torch.Size([input.size(0), input.size(1), self.hidden_dim]) # batch, seq_len, hidden

        
        # Run activation and get outputs
        lin_act = self.linear(lstm_out)
        # Expect [batch_size, seq_len, 1]
        assert lin_act.shape == torch.Size([input.size(0), input.size(1), 1])
        return lin_act


model = LSTM(input_dim = lstm_input_size,
             hidden_dim=hidden_dim,
             output_dim=1,
             num_layers=num_layers)

In [ ]:
l1_loss = nn.L1Loss()
    
optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimiser, step_size=lr_decay_step_size, gamma=lr_gamma)
#####################---------------------------------------------------------------------------
# Train model
#####################

loss_history = []
val_loss_history = []

best_loss = 1000
for t in range(num_epochs):
    epoch_loss = 0
    val_epoch_loss = 0
    total_items = 0
    val_total_items = 0
    # List to store all predictions
    all_train_pred = []
    all_val_pred = []
    for x_batch, y_batch in train_loader: 
        
        # Clear stored gradient
        optimiser.zero_grad()
        
        # Training: Predict and calc loss
        train_pred = model(x_batch)
        loss = l1_loss(train_pred.view(x_batch.size(0), -1), y_batch)
        all_train_pred.append(train_pred)
        
        # Tracking mean loss across batches
        epoch_loss += train_pred.shape[0]*loss.item()
        total_items += train_pred.shape[0]
        
        
#         # Validation: predict and calc loss
#         val_pred = model(val_x_batch)
#         # Flatten to (batch_size*timesteps) x 4
#         val_pred = val_pred.view(val_pred.shape[0]*val_pred.shape[1], 4)
#         all_val_pred.append(val_pred)
#         val_loss = zoib.zoib_loss(
#             val_pred, 
#             val_y_batch.view(val_y_batch.shape[0]*val_y_batch.shape[1], 1)[:,0]  # Flatten to (batch_size*timesteps) X 1
#         ).float()

#         # Tracking mean loss across batches
#         val_epoch_loss += val_pred.shape[0]*val_loss.item()
#         val_total_items += val_pred.shape[0]


        # Backward pass
        loss.backward()
        
        # Update parameters
        optimiser.step()

    # LR decay
    scheduler.step()
    
    epoch_loss = epoch_loss/total_items
#     val_epoch_loss = val_epoch_loss/val_total_items
    loss_history.append(epoch_loss)
#     val_loss_history.append(val_epoch_loss)
    if t%10==0:
        print("Epoch ", t, "Train Loss: ", epoch_loss, "LR: ", optimiser.param_groups[0]["lr"])
        
    if np.isnan(epoch_loss):
        break
        
    # Early stopping
    if epoch_loss < best_loss:
        i = 0
        # Save best loss, predictoins, and hidden state
        best_loss = epoch_loss
        best_train_pred = torch.cat(all_train_pred, dim=0)
#         best_val_pred = torch.cat(all_val_pred, dim=0)    
    if (i > early_stopping):
        break

# View results

In [ ]:
plt.plot(loss_history)

In [ ]:
plt.scatter(train_y.flatten(), best_train_pred.detach().numpy().flatten())
plt.xlabel('True')
plt.ylabel('Pred')
plt.show()

In [ ]:
pd.DataFrame({'Pred':best_train_pred.detach().numpy().flatten(), 'True':train_y.flatten()}).plot()